In [1]:
  #!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import numpy as np
from astropy.io import fits
from astropy.table import Table
import sys
sys.path.insert(0,'/home/henrique/Programas_Cosmologia/CLCosmo_Sim/cluster_abundance/')
sys.path.insert(0,'../../scripts')

import pyccl as ccl
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_cluster_abundance as cl_count
import time
from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

initialize the `Cosmological model`

In [2]:
cosmo = Nc.HICosmo.new_from_name(Nc.HICosmo, "NcHICosmoDEXcdm")
reion = Nc.HIReionCamb.new () 

prim = Nc.HIPrimPowerLaw.new () 

cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFunc.new_from_name('NcTransferFuncEH')

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-3)
psml.require_kmax (1.0e3)
psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()

mulf = Nc.MultiplicityFuncBocquet.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)
mulf.set_sim (Nc.MultiplicityFuncBocquetSim.DM)

hmf = Nc.HaloMassFunction.new (dist, psf, mulf)
hmf.set_area((0.0005)*np.pi)

#Creating the halo bias 
hbias_ps = Nc.HaloBiasTypePS()
mbiasf = Nc.HaloBiasFunc.new(hmf, hbias_ps)

# Define proxy modelling
Use a mass proxy, define the probability for observing a proxy given a mass and redhsift

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
    \mu(M,z) = \mu_0 + a_\mu^M\log_{10}\frac{M}{M_0} + a_\mu^z\log_{10}\frac{1+z}{1+z_0} 
$$
variance is
$$
\sigma(M,z) = \sigma_0 + a_\sigma^M\log_{10}\frac{M}{M_0} + a_\sigma ^z\log_{10}\frac{1+z}{1+z_0} 
$$

In [3]:
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (0,2))
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, 0, np.log(10) * np.log10(10e6)))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))

Create the `Cluster Abundance Data`

In [4]:
ca = Nc.ClusterAbundance.new(hmf,mbiasf)
mset = Ncm.MSet.new_array([cosmo,cluster_m,cluster_z])

#Generates cluster count catalog data


ncdata = Nc.DataClusterNCount.new(ca,'NcClusterRedshiftNodist','NcClusterMassAscaso')
rng = Ncm.RNG.pool_get("example_ca_sampling");
ncdata.init_from_sampling(mset,(0.0005)*np.pi,rng)
ncdata.catalog_save("ca_mean_bias_ascaso.fits", True)

ncdata_fits = fits.open('ca_mean_bias_ascaso.fits')
#ncdata_fits.info()

ncdata_data =  ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table.columns
ncdata_Table.sort('Z_TRUE')
mset.pretty_log()


#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.36               [FIXED]
#   -     Omegac[01]:  0.2568              [FIXED]
#   -     Omegax[02]:  0.7                 [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0432              [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#----------------------------------------------------------------------------------
# Model parameters
#   - ln10e10ASA[00]:  3.179               [FIXED]
#   - T_SA_ratio[01]:  0.2               

# Un-binned approach

$$
x_i = \int_{\log_{10}M_{min}}^{\log_{10}M_{max}}\frac{d^2V(z_i)}{dz d\Omega}\frac{dn(M,z_i)}{d\log_{10}M}P(\lambda_i|M,z_i)b(M,z_i) d\log_{10}M
$$

In [6]:
display(ncdata_Table)
ca.prepare(cosmo,cluster_z,cluster_m)


print(ca.mean_bias(cosmo, cluster_z, cluster_m))
dnbias = []
i = 0
for cluster in ncdata_Table:
    i +=1
    dnbias.append(ca.intp_d2n_bias(cosmo, cluster_z, cluster_m, [cluster[1]], None, [cluster[0]], None ))

Z_OBS,LNM_OBS,Z_TRUE,LNM_TRUE
float64,float64,float64,float64
0.06995504829395446,0.9787360327553738,0.06995504829395446,31.082030869650588
0.08350670997616837,1.6422895444599972,0.08350670997616837,31.753855188773095
0.08758075309927918,1.954219370855404,0.08758075309927918,32.19291522623267
0.0969605107435312,1.0220202019128672,0.0969605107435312,31.071930326385534
0.10564334748414914,0.7020793355277206,0.10564334748414914,31.124717691071613
0.1068233309705027,0.9610714904929332,0.1068233309705027,30.008018812923613
0.11765674012388246,0.19760626356254804,0.11765674012388246,30.251392268706617
0.11914761034453737,0.08659748228875613,0.11914761034453737,30.54242622681923
0.12114714692400483,1.0135078651866318,0.12114714692400483,31.819603270087985


: 

: 